I. 사용자 주가 등락 예측(익일) 서비스는 당일 오전 12시부터 오후 11시 59분 59초까지 사용가능.
    i) 월요일 주가 등락 예측의 경우 전주 금요일 오전 12시부터 일요일 오후 11시 59분 59초까지 사용가능. 
II. 오전 12시가 되면 전날 기사를 크롤링하는 것으로 함.
    i) 주말이 끼어있을 경우, 월요일 오전 12시가 되면 전주 금,토,일 기사를 모두 크롤링하는 것으로 함., 월요일 오전 12시가 되면 전주 금,토,일 기사를 모두 크롤링하는 것으로 함.
III. 크롤링 된 기사로 기업명 키워드 랭크10(A)을 뽑아내고, 해당 기업명이 들어가 있는 문장(B)을 전날 크롤링한 기사에서 뽑아냄
IV. 뽑아낸 문장(B)을 fune-tuning된 kobert 모델을 활용하여 (2,1,0으로) 라벨링함
V. 라벨링된 값으로 각 기업(A)의 점수를 계산
VI. 계산된 점수를 바탕으로 expect(up,stay,down)을 라벨링
VII. expect(up,stay,down)를 바탕으로 recommend(buy, stay, sell)을 라벨링

In [1]:
from settingDatetime import *

In [2]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, tqdm_notebook

In [3]:
import easydict
import pandas as pd

In [4]:
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [5]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair) 

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]
        #self.sentences = [transform([i]) for i in dataset[dataset.columns[sent_idx]]]
        #self.labels = [np.int32(i) for i in dataset[dataset.columns[label_idx]]]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [6]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=3, # 3가지로 분류
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [7]:
# sentences : 예측하고자 하는 텍스트 데이터 리스트
def getLabelValue(sentences, tok, max_len, batch_size, device, model):
    textList = [] # 텍스트 데이터를 담을 리스트
    labelList = [] # 라벨을 담을 리스트
    for s in sentences: # 모든 문장
        textList.append([s,5]) # [문장, 임의의 양의 정수값] 설정
    
    # print(textList)
    pdData = pd.DataFrame(textList, columns=[['text', 'label']])
    pdData = pdData.values
    test_set = BERTDataset(pdData, 0, 1, tok, max_len, True, False)
    test_input = torch.utils.data.DataLoader(test_set, batch_size=batch_size, num_workers=5)

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        out = model(token_ids, valid_length, segment_ids)
        #print(out)
        for s in out:
            if (s[0]>=s[1])&(s[0]>=s[2]):
                value = -1
            elif (s[1]>s[0])&(s[1]>s[2]):
                value = 0
            else:
                value = 1
            labelList.append(value)
    
    result = pd.DataFrame({'text':sentences,'label':labelList})
    
    return result


In [8]:
if __name__ == "__main__":
    
    # 파라미터1 데이터셋 불러오기
    data = pd.read_csv(f'./data/input/{target}_input.csv',encoding='utf-8-sig')
    word = data['word']
    input_data = list(data['text'])
    
    
    # 파라미터2 tok 설정
    bertmodel, vocab = get_pytorch_kobert_model()
    ## 기본 Bert tokenizer 사용
    tokenizer = get_tokenizer()
    tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)
    
    
    # 파라미터5 device 설정 
    ## CPU
    if torch.cuda.is_available()==False:
        device = torch.device("cpu")
    ## GPU
    else:
        device = torch.device("cuda:0")
    #print(torch.cuda.is_available())
    
    # 파라미터6 모델 불러오기
    ## 모델 클래스는 어딘가에 반드시 선언되어 있어야 한다.
    model = torch.load("./model/model_cpu.pt")
    model.eval()
    
    #jupter notebook에서는 사용할 수 없음
    #parser = argparse.ArgumentParser(description='Process some integers.')

    #parser.add_argument("--tok", type=nlp.data.transforms.BERTSPTokenizer, default=tok)
    #parser.add_argument("--max_len", type=int, default=200) # 해당 길이를 초과하는 단어에 대해선 bert가 학습하지 않음
    #parser.add_argument("--batch_size", type=int, default=16)
    #parser.add_argument("--device", type=torch.device, default=device)
    #parser.add_argument("--checkpoint_path", type=str, default="./")
    #arges = parser.parse_args()
    
    args = easydict.EasyDict({
        "sentences" : input_data,
        "tok" : tok,
        "max_len" : 200,
        "batch_size" : 16,
        "device" : torch.device("cpu"), # default = device
        "model" : model
    })

    # get output data
    output = getLabelValue(args.sentences, args.tok, args.max_len, args.batch_size, args.device, args.model)
    output = pd.concat([data['date'],data['day'],word,output],axis=1)
    output.to_csv(f'./data/output/{target}_output.csv',encoding='utf-8-sig',index=False)
    display(output)

using cached model. /home/kongminseok/hdd/2022_kbSecurities_DIM/.cache/kobert_v1.zip
using cached model. /home/kongminseok/hdd/2022_kbSecurities_DIM/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/kongminseok/hdd/2022_kbSecurities_DIM/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


<ipython-input-7-5f10bfeea6ff>:14: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm_notebook(test_input)):


  0%|          | 0/18 [00:00<?, ?it/s]

,date,day,word,text,label
0,20220325-20220327,"금,토,일",삼성,"삼성家, 전자 지분 1조3천억'블록딜'",1
1,20220325-20220327,"금,토,일",삼성,삼성증권 '美주식 주간거래' 한달만에 5000억 돌파,1
2,20220325-20220327,"금,토,일",삼성,"삼성물산, 삼성전자와 8406억원 규모 평택사업장 공사 수주",1
3,20220325-20220327,"금,토,일",삼성,"삼성증권, 美주식 주간 거래 서비스 인기…한달새 5000억 돌파",1
4,20220325-20220327,"금,토,일",삼성,삼성전자 '6만전자'로 전락…12조 상속세 때문에 1조 넘게 블록딜 처분,1
...,...,...,...,...,...
272,20220325-20220327,"금,토,일",제과,[특징주] 롯데제과·롯데푸드 합병 결정에 동반 강세,1
273,20220325-20220327,"금,토,일",제과,[특징주] 롯데제과·롯데푸드 합병에 양사 주가 강세,1
274,20220325-20220327,"금,토,일",제과,"[특징주] 롯데제과, 롯데푸드와 합병 소식에 4%↑",1
275,20220325-20220327,"금,토,일",제과,[특징주]롯데제과·푸드 합병 결정에 강세,1


In [9]:
dic = {'date' : target,'days': target_day,'word':[], 'score':[], 'expect':[],'recommend':[]}
for company in output.word.unique():
    score = output[output.word==company].label.sum()
    if score>0:
        expect = 'up'
        recommend = 'buy'
    elif score==0:
        expect = 'stay'
        recommend =  'stay'
    elif score<0:
        expect = 'down'
        recommend = 'sell'
    dic['word'].append(company)
    dic['score'].append(score)
    dic['expect'].append(expect)
    dic['recommend'].append(recommend)
result = pd.DataFrame(dic)
result = result.sort_values('score',ascending=False)
result.to_csv(f'./data/result/{target}_result.csv',encoding='utf-8-sig',index=False)
display(result)

,date,days,word,score,expect,recommend
0,20220325-20220327,"금,토,일",삼성,71,up,buy
1,20220325-20220327,"금,토,일",안랩,39,up,buy
3,20220325-20220327,"금,토,일",블록딜,32,up,buy
2,20220325-20220327,"금,토,일",롯데,26,up,buy
4,20220325-20220327,"금,토,일",두산,24,up,buy
5,20220325-20220327,"금,토,일",미래에셋,21,up,buy
6,20220325-20220327,"금,토,일",박용만,17,up,buy
7,20220325-20220327,"금,토,일",최현,16,up,buy
8,20220325-20220327,"금,토,일",푸드,16,up,buy
9,20220325-20220327,"금,토,일",제과,15,up,buy
